# Embedding
컴퓨터는 자연어를 직접 처리할 수 없기에 숫자나 벡터 형태로 변환할 필요가 있다. 이를 **임베딩(Embedding)**이라고 한다. 임베딩은 말뭉치의 의미에 따라 벡터화하기 때문에 문법적인 정보가 포함되어 있다. 임베딩 기법에는 문장 임베딩과 단어 임베딩이 있다.

**문장 임베딩은 문장 전체를 벡터로 표현하는 방법이며, 단어 임베딩은 개별 단어를 벡터로 표현하는 방법**이다. **문장임베딩의 경우 전체 문장의 흐름을 파악해 벡터로 변환하기 때문에 문맥적 의미를 지니는 장점**이 있다. 그런 이유로 단어 임베딩에 비해 품질이 좋으며, 상용시스템에 많이 사용된다. 하지만 임베딩하기 위해 **문장 데이터가 많이 필요하며 학습하는데 비용이 많이 들어간다**.

단어임베딩은 동음이의어에 대한 구분을 하지 않기 때문에 의미가 다르더라도 단어의 형태가 같다면 동일한 벡터값으로 표현되는 단점이 있다. 하지만 문장 임베딩에 비해 간단하기 때문에 여전히 실무에서 많이 사용한다. 

## 단어 임베딩
### One-Hot Encoding

**원-핫 인코딩(One-Hot Encoding)은 단어를 숫자 벡터로 변환하는 가장 기본적인 방법이다. 명칭에서도 알 수 있듯이 요소들 중 단 하나의 값만 1이고 나머지 요솟값은 0인 인코딩을 의미한다.** **원-핫 인코딩(One-Hot Encoding)으로 나온 결과를 One-Hot Vector**라고 하며, 전체 요소 중 단 하나의 값만 1이기 때문에 **희소(sparse) 벡터**라고 한다.

원-핫 인코딩을 하기 위해서는 단어 집합이라 불리는 사전을 먼저 만들어야 한다. 여기서 사전은 말뭉치에서 나오는 서로 다른 모든 단어의 집합을 의미한다. 말뭉치에 존재하는 모든 단어의 수가 원-핫 벡터의 차원을 결정한다.

In [2]:
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 1.3 MB/s 
     |████████████████████████████████| 453 kB 37.6 MB/s 


In [3]:
from konlpy.tag import Komoran
import numpy as np

komoran = Komoran()
text = "이렇게 좋아하는 사람을 만나는 계절은 두 번 다시 없어요." # 아라시 Love so sweet 가사 中

# 명사만 추출
nouns = komoran.nouns(text)
print(nouns)

# 단어 사전 구축 및 단어별 인덱스 부여
dics = {}
for word in nouns :
  if word not in dics.keys():
    dics[word] = len(dics)
print(dics)

# One-Hot Encoding
nb_classes=len(dics)
targets= list(dics.values())
one_hot_targets = np.eye(nb_classes)[targets] # 원-핫 벡터를 만들기 위해서는 Numpy의 eye()함수를 이용한다. 
# eye()함수는 단위 행렬을 만들어준다. eye() 함수의 인자 크기대로 단위행렬을 반환하며, eye() 함수 뒤에 붙은 [targets]를 이용해 생성된 단위 행렬의 순서를 단어 사전의 순서에 맞게 정렬해준다.
print(one_hot_targets)

['사람', '계절', '번']
{'사람': 0, '계절': 1, '번': 2}
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


## 희소 표현
위처럼 단어의 인덱스 요소만 1이고 나머지 요소는 모두 0으로 표현되는 희소 벡터(희소 행렬)로 단어를 표현하는 방식을 **희소 표현(sparse representation)**이라고 한다.

**희소 표현은 각각의 차원이 독립적인 정보를 지니고 있어 사람이 이해하기에 직관적인 장점이 있지만 단어 사전의 크기가 커질수록 메모리 낭비와 계산 복잡도가 커지는 단점**이 있다. 또한 단어 간의 연관성이 전혀 없어 의미를 담을 수 없다.

자연어 처리를 잘하기 위해서는 기본 토큰이 되는 단어의 의미와 주변 단어 간의 관계가 단어 임베딩에 표현되어야 한다. 이를 위해 각 단어 간의 유사성을 잘 표현하면서도 벡터 공간을 절약할 수 있는 방법을 고안했는데, **이를 분산 표현(distributed representation)이라고 한다. 분산 표현은 하나의 차원에 다양한 정보를 가지고 있다. **RGB 모델은 3차원 형태의 벡터로 분산 표현의 방식 중 하나이다. 

신경망에서는 분산 표현을 학습하는 과정에서 임베딩 벡터의 모든 차원에 의미있는 데이터를 고르게 밀집시키며, 이로 인해 데이터 손실이 최소화되고 벡터 차원이 압축되는 효과가 생긴다. **분산표현 방식은 우리가 원하는 차원에 데이터를 최대한 밀집시킬 수 있어 밀집 표현(Dense Representation)이라 부르기도 하며, 밀집 표현으로 만들어진 벡터를 밀집 벡터(dense vector)라 한다.**

입력 데이터의 차원이 너무 높아지면 싱경망 모델의 학습이 어려워지는 차원의 저주(curse of dimensionality) 문제가 발생한다. 분산 표현은 이를 방지하며, 단어의 의미, 주변 단어 간의 관계 등 많은 정보가 내포되어 일반화 능력이 뛰어나다. 

# Word2Vec
Word2Vec은 2013년 구글에서 발표했으며 가장 많이 사용하고 있는 단어 임베딩 모델이다. 기존 신경망 기반의 단어 임베딩 모델에 비해 구조상 차이는 크게 없지만 계산량을 획기적으로 줄여 빠른 학습을 가능하게 했다. Word2Vec 모델은 CBOW(continuous bag-of-words)와 skip-gram 두 가지 모델로 제안되었다.

## CBOW 모델
맥락(context word)이라 표현되는 주변 단어들을 이용해 타깃 단어를 예측하는 신경망 모델이다. 신경망의 입력을 주변 단어들로 구성하고 출력을 타깃 단어로 설정해 학습된 가중치 데이터를 임베딩 벡터로 활용한다.

## skip-gram 모델
CBOW 모델과 반대로 하나의 타깃 단어를 이용해 주변 단어들을 예측하는 신경망 모델이다. 입출력이 CBOW 모델이랑 반대로 되어 있기 때문에 skip-gram 모델이 CBOW 모델에 비해 예측해야 하는 맥락이 많아진다. 따라서 단어 분산 표현력이 우수해 CBOW 모델에 비해 임베딩 품질이 우수하다.
반면 CBOW 모델은 타깃 단어의 손실만 계산하면 되기 때문에 학습속도가 빠른 장점이 있다.

Word2Vec의 단어 임베딩은 해당 단어를 밀집 벡터로 표현하며 학습을 통해 의미상 비슷한 단어들을 비슷한 벡터 공간에 위치한다. 또한 벡터 특성상 의미에 따라 방향성을 갖게 되는데, 임베딩된 벡터들 간 연산이 가능하기 때문에 단어 간 관계를 계산할 수 있다.